# Atrial Fibrillation Features

As with activity classification, we will featurize our time series signal and throw those features into a classifier. For AF we will build a two-class classifier using the inter-beat-interval time series. This time series can be derived by taking the difference between successive QRS complex locations as provided by the Pan-Tompkins algorithm in the previous videos. We source our features from a couple submissions to the [Computing in Cardiology Challenge 2017](https://physionet.org/content/challenge-2017/1.0.0/). 

>[Behar, Rosenberg, Yaniv, Oster. Rhythm and Quality Classification from Short ECGs Recorded Using a Mobile Device. Computing in Cardiology Challenge 2017.](http://www.cinc.org/archives/2017/pdf/165-056.pdf)  

and   
>[Bonizzi, Driessens, Karel. Detection of Atrial Fibrillation Episodes from Short Single Lead Recordings by Means of Ensemble Learning. Computing in Cardiology Challenge 2017.](http://www.cinc.org/archives/2017/pdf/169-313.pdf)


Recall that the RR interval time series is irregularly sampled because we get a new measurement at each heart beat, instead of at some fixed interval in time. Many of the features we want can be computed on an irregular time series, but some, especially frequency domain features, cannot. In that case, we will first have to make this uniform by using interpolation.


## Imports

In [1]:
import glob
import os

import numpy as np
import pandas as pd

## Load Data

In [2]:
fs = 300
data_dir = 'cinc/'
ref = pd.read_csv(data_dir + 'REFERENCE.csv')
ref = dict(zip(ref.record, ref.rhythm))
base = lambda f: os.path.splitext(os.path.basename(f))[0]
files = sorted(glob.glob(data_dir + '*.npz'))
qrs = []
labels = []
for f in files:
    with np.load(f) as npz:
        qrs.append(npz['qrs'])
    labels.append(ref[base(f)])

## Features

The features for the AF detection algorithm are computed from the RR interval time series. We use the time domain and frequency domain features listed below.

### Time domain
 - minimum RR interval
 - maximum RR interval
 - median RR interval
 - average RR interval
 - standard deviation of RR intervals
 - number of RR interval outliers
   - An RR interval is an outlier if it is greater than 1.2x the average RR interval in the ECG record

 - root-mean-square of the difference between adjacent RR intervals
 - percent of RR interval differences greater than 50 milliseconds

### Frequency domain
The RR interval time series is not sampled regularly in time. We only have a datapoint every heart beat. Before we can compute any frequency domain features, the time series must be resampled so that we get uniform data points. Resample the RR interval time series to 4 Hz before computing the features below.

 - peak magnitude between 0.04 Hz and 0.15 Hz in the regularized RR interval time series
 - main frequency between 0.04 Hz and 0.15 Hz in the regularized RR interval time series
 - peak magnitude between 0.15 Hz and 0.4 Hz in the regularized RR interval time series
 - main frequency between 0.15 Hz and 0.4 Hz in the regularized RR interval time series

In [3]:
def Featurize(qrs_inds, fs):
    """Featurize the qrs complex locations time series.

    Args:
        qrs_inds: (np.array of number) the sample indices of the QRS complex locations
        fs: (number) the sampling rate

    Returns:
        n-tuple of features
    """
    # Compute the RR interval time series
    rr = np.diff(qrs_inds)

    # Compute time domain features
    if len(qrs_inds) < 1:
        return [0,0,0,0,0,0,0,0,0,0,0,0]
    else:
        pass
    min_rr = np.min(rr)
    max_rr = np.max(rr)
    median_rr = np.median(rr)
    mean_rr = np.mean(rr)
    std_rr = np.std(rr)
    n_outliers = np.sum(rr > 1.2 * np.mean(rr))
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr))))
    pdrr_50 = np.mean(np.diff(rr) / fs > 0.05)

    # Regularly resample the RR interval time series
    rr_ts = np.arange(qrs_inds[0] / fs, qrs_inds[-2] / fs, 1 / 4)
    rr_interp = np.interp(rr_ts, qrs_inds[:-1] / fs, rr)

    # Compute the Fourier transform of the regular RR interval time series
    freq = np.fft.rfftfreq(len(rr_interp), 1 / 4)
    fft_mag = np.abs(np.fft.rfft(rr_interp))

    # Compute frequency domain features
    lf_mag = np.max(fft_mag[(freq >= 0.04) & (freq <= 0.15)])
    lf_freq = freq[np.argmax(fft_mag[(freq >= 0.04) & (freq <= 0.15)])]
    hf_mag = np.max(fft_mag[(freq >= 0.15) & (freq <= 0.4)])
    hf_freq = freq[np.argmax(fft_mag[(freq >= 0.15) & (freq <= 0.4)])]


    return (min_rr, max_rr, median_rr, n_outliers, mean_rr, std_rr, rmssd, pdrr_50,
            lf_mag, lf_freq, hf_mag, hf_freq)

## Create Feature Matrix

In [4]:
feats = [Featurize(qrs_inds, fs) for qrs_inds in qrs]      
X, y = np.array(feats), np.array(labels)

## Build a Classifier

In [5]:
# TODO:
# Write code that builds and trains a classifier to classify our ECG records.
# Again, a random forest with 100 trees and a depth of 4 works well.
# Evaluate the performance of the classifier using cross validation.

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

# Use A RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,
                             max_depth=4,
                             random_state=0,
                             class_weight='balanced')

# Evaluate with 7-fold cross validation
cv = KFold(7, shuffle=True, random_state=0)

# Keep an aggregated confusion matrix
cm = np.zeros((2,2), dtype='int')

# As well as aggregated predictions with labels
y_tests = []
y_preds = []

for train_ind, test_ind in cv.split(X, y):
    # Subset dataset
    X_train, y_train = X[train_ind], y[train_ind]
    X_test, y_test = X[test_ind], y[test_ind]
    
    # Train model
    clf.fit(X_train, y_train)
    
    # Run model
    y_pred = clf.predict(X_test)
    
    # Aggregate results
    y_tests.append(y_test)
    y_preds.append(y_pred)
    cm += confusion_matrix(y_test, y_pred, labels=['O', 'N'])
    

## Compute Performance Metrics
We will evaluate the classfier performance by computing precision and recall on detecting no-sinus(other) rhythm


## Using aggregated predictions and labels

From a vector of all predictions and the correct labels, we can use numpy operations to count the 
number of true positives, false positives, and false negatives. We can then use this to compute and recall


In [7]:
yt = np.hstack(y_tests)
yp = np.hstack(y_preds)
tp = np.sum((yp == 'O') & (yt == 'O'))
fp = np.sum((yp == 'O') & (yt == 'N'))
fn = np.sum((yp == 'N') & (yt == 'O'))
precision = tp / (tp + fp)
recall = tp / (tp + fn)
precision, recall


(0.7262569832402235, 0.8105299618981642)

## Using the confusion matrix

We can also get the number of true positves, false positives, and false negatives directly from the confusion matrix to compute precison and recall

In [8]:
tp = cm[0, 0]
fp =  cm[1, 0]
fn = cm[0, 1]
precision = tp / (tp + fp)
recall = tp / (tp + fn)
precision, recall

(0.7262569832402235, 0.8105299618981642)

## Summary
I used a RandomForestClassifier with 100 trees and a max depth of 4. With these basic features, it achieved a precision of 73% and a recall of 81%. Further improvements can be made by using a more advanced technique for RR interval time series regularization, and by using more informative features. See the further resources sections for example of more advanced atrial fibrillation detection algorithms.

